In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.12.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\jihyu\anaconda3\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.1568, train_loss: 0.43391725, train_accuracy: 0.2500, test_Accuracy: 0.1367
Epoch: [ 0] [    1/  468] time: 1.6625, train_loss: 0.38465184, train_accuracy: 0.5234, test_Accuracy: 0.3959
Epoch: [ 0] [    2/  468] time: 2.1962, train_loss: 0.39543191, train_accuracy: 0.4609, test_Accuracy: 0.4720
Epoch: [ 0] [    3/  468] time: 2.6819, train_loss: 0.35463619, train_accuracy: 0.4922, test_Accuracy: 0.4878
Epoch: [ 0] [    4/  468] time: 3.1857, train_loss: 0.33765778, train_accuracy: 0.6250, test_Accuracy: 0.5102
Epoch: [ 0] [    5/  468] time: 3.6322, train_loss: 0.36638537, train_accuracy: 0.5547, test_Accuracy: 0.5643
Epoch: [ 0] [    6/  468] time: 4.0816, train_loss: 0.31826323, train_accuracy: 0.6172, test_Accuracy: 0.6418
Epoch: [ 0] [    7/  468] time: 4.5880, train_loss: 0.30103782, train_accuracy: 0.7188, test_Accuracy: 0.7123
Epoch: [ 0] [    8/  468] time: 5.1347, train_loss: 0.35562089, train_accuracy: 0.7578, test_Accuracy: 0.7563
Epoch: [ 0

Epoch: [ 0] [   74/  468] time: 38.7007, train_loss: 0.13062531, train_accuracy: 0.9453, test_Accuracy: 0.9365
Epoch: [ 0] [   75/  468] time: 39.2393, train_loss: 0.15392652, train_accuracy: 0.9531, test_Accuracy: 0.9378
Epoch: [ 0] [   76/  468] time: 39.7872, train_loss: 0.22362781, train_accuracy: 0.9297, test_Accuracy: 0.9409
Epoch: [ 0] [   77/  468] time: 40.4386, train_loss: 0.17411119, train_accuracy: 0.9531, test_Accuracy: 0.9416
Epoch: [ 0] [   78/  468] time: 41.1393, train_loss: 0.11672661, train_accuracy: 0.9375, test_Accuracy: 0.9415
Epoch: [ 0] [   79/  468] time: 41.7445, train_loss: 0.11457048, train_accuracy: 0.9453, test_Accuracy: 0.9399
Epoch: [ 0] [   80/  468] time: 42.2575, train_loss: 0.11196789, train_accuracy: 0.9453, test_Accuracy: 0.9413
Epoch: [ 0] [   81/  468] time: 42.7777, train_loss: 0.12490253, train_accuracy: 0.9531, test_Accuracy: 0.9412
Epoch: [ 0] [   82/  468] time: 43.2648, train_loss: 0.16228837, train_accuracy: 0.9531, test_Accuracy: 0.9407
E

Epoch: [ 0] [  148/  468] time: 76.8294, train_loss: 0.07200950, train_accuracy: 0.9844, test_Accuracy: 0.9523
Epoch: [ 0] [  149/  468] time: 77.4145, train_loss: 0.13195015, train_accuracy: 0.9688, test_Accuracy: 0.9517
Epoch: [ 0] [  150/  468] time: 77.9228, train_loss: 0.10564970, train_accuracy: 0.9766, test_Accuracy: 0.9506
Epoch: [ 0] [  151/  468] time: 78.4413, train_loss: 0.13037099, train_accuracy: 0.9609, test_Accuracy: 0.9507
Epoch: [ 0] [  152/  468] time: 78.9729, train_loss: 0.10332360, train_accuracy: 0.9844, test_Accuracy: 0.9514
Epoch: [ 0] [  153/  468] time: 79.5012, train_loss: 0.09028791, train_accuracy: 0.9688, test_Accuracy: 0.9520
Epoch: [ 0] [  154/  468] time: 79.9994, train_loss: 0.15703282, train_accuracy: 0.9453, test_Accuracy: 0.9523
Epoch: [ 0] [  155/  468] time: 80.4829, train_loss: 0.09203095, train_accuracy: 0.9766, test_Accuracy: 0.9512
Epoch: [ 0] [  156/  468] time: 80.9717, train_loss: 0.16693607, train_accuracy: 0.9297, test_Accuracy: 0.9515
E

Epoch: [ 0] [  222/  468] time: 114.6811, train_loss: 0.06285440, train_accuracy: 0.9766, test_Accuracy: 0.9571
Epoch: [ 0] [  223/  468] time: 115.1747, train_loss: 0.09881769, train_accuracy: 0.9609, test_Accuracy: 0.9565
Epoch: [ 0] [  224/  468] time: 115.6599, train_loss: 0.07164267, train_accuracy: 0.9531, test_Accuracy: 0.9562
Epoch: [ 0] [  225/  468] time: 116.1652, train_loss: 0.06971375, train_accuracy: 0.9688, test_Accuracy: 0.9557
Epoch: [ 0] [  226/  468] time: 116.6522, train_loss: 0.14482102, train_accuracy: 0.9453, test_Accuracy: 0.9546
Epoch: [ 0] [  227/  468] time: 117.1429, train_loss: 0.10408893, train_accuracy: 0.9688, test_Accuracy: 0.9550
Epoch: [ 0] [  228/  468] time: 117.7129, train_loss: 0.05378595, train_accuracy: 0.9688, test_Accuracy: 0.9552
Epoch: [ 0] [  229/  468] time: 118.2177, train_loss: 0.10668424, train_accuracy: 0.9531, test_Accuracy: 0.9557
Epoch: [ 0] [  230/  468] time: 118.7560, train_loss: 0.07960083, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 152.9267, train_loss: 0.03260227, train_accuracy: 0.9766, test_Accuracy: 0.9588
Epoch: [ 0] [  297/  468] time: 153.6294, train_loss: 0.03083190, train_accuracy: 0.9766, test_Accuracy: 0.9593
Epoch: [ 0] [  298/  468] time: 154.1151, train_loss: 0.05619622, train_accuracy: 0.9844, test_Accuracy: 0.9595
Epoch: [ 0] [  299/  468] time: 154.6153, train_loss: 0.07495669, train_accuracy: 0.9766, test_Accuracy: 0.9588
Epoch: [ 0] [  300/  468] time: 155.1357, train_loss: 0.06817010, train_accuracy: 0.9688, test_Accuracy: 0.9591
Epoch: [ 0] [  301/  468] time: 155.7099, train_loss: 0.05489318, train_accuracy: 0.9766, test_Accuracy: 0.9586
Epoch: [ 0] [  302/  468] time: 156.2573, train_loss: 0.10740522, train_accuracy: 0.9375, test_Accuracy: 0.9600
Epoch: [ 0] [  303/  468] time: 156.7614, train_loss: 0.04635304, train_accuracy: 0.9922, test_Accuracy: 0.9609
Epoch: [ 0] [  304/  468] time: 157.2602, train_loss: 0.08410472, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 196.0720, train_loss: 0.06594901, train_accuracy: 0.9609, test_Accuracy: 0.9538
Epoch: [ 0] [  371/  468] time: 196.5895, train_loss: 0.02670945, train_accuracy: 1.0000, test_Accuracy: 0.9541
Epoch: [ 0] [  372/  468] time: 197.1936, train_loss: 0.03119848, train_accuracy: 0.9766, test_Accuracy: 0.9543
Epoch: [ 0] [  373/  468] time: 197.8095, train_loss: 0.04895830, train_accuracy: 0.9688, test_Accuracy: 0.9568
Epoch: [ 0] [  374/  468] time: 198.3614, train_loss: 0.07704861, train_accuracy: 0.9531, test_Accuracy: 0.9583
Epoch: [ 0] [  375/  468] time: 198.9110, train_loss: 0.07085320, train_accuracy: 0.9922, test_Accuracy: 0.9594
Epoch: [ 0] [  376/  468] time: 199.4386, train_loss: 0.06763180, train_accuracy: 0.9844, test_Accuracy: 0.9601
Epoch: [ 0] [  377/  468] time: 200.0073, train_loss: 0.10688168, train_accuracy: 0.9609, test_Accuracy: 0.9624
Epoch: [ 0] [  378/  468] time: 200.5258, train_loss: 0.12377748, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 238.7213, train_loss: 0.06998093, train_accuracy: 0.9531, test_Accuracy: 0.9716
Epoch: [ 0] [  445/  468] time: 239.2466, train_loss: 0.06643707, train_accuracy: 0.9844, test_Accuracy: 0.9720
Epoch: [ 0] [  446/  468] time: 239.7994, train_loss: 0.05193496, train_accuracy: 0.9844, test_Accuracy: 0.9714
Epoch: [ 0] [  447/  468] time: 240.3794, train_loss: 0.09316613, train_accuracy: 0.9688, test_Accuracy: 0.9708
Epoch: [ 0] [  448/  468] time: 240.9154, train_loss: 0.05904163, train_accuracy: 0.9844, test_Accuracy: 0.9705
Epoch: [ 0] [  449/  468] time: 241.5596, train_loss: 0.09166145, train_accuracy: 0.9844, test_Accuracy: 0.9701
Epoch: [ 0] [  450/  468] time: 242.2885, train_loss: 0.04639229, train_accuracy: 0.9922, test_Accuracy: 0.9699
Epoch: [ 0] [  451/  468] time: 242.9283, train_loss: 0.08381256, train_accuracy: 0.9375, test_Accuracy: 0.9701
Epoch: [ 0] [  452/  468] time: 243.5603, train_loss: 0.04946509, train_accuracy: 0.9922, test_Accuracy: